If you are like me and have limited space on your laptop, this notebook will help you alot. I wanted to get a larger training file that I could use for building models on my laptop and then testing it on that small subset and move it to the main training on AWS. Besides, I have a limited internet connection. In this notebook, I'm generating a larger training sample than the one provided. Allowing me to build and test algorithms before moving it to larger AWS machine to excute on the entire dataset. Through testing Kaggle file limits, I found that 40MB is a good size acceptable by the Notebook environment here on Kaggle and 5MB for test. I wish I was able to make it about 1GB for train and about 100MB for testing. That would have been a better.

In [ ]:
import bson 
from bson import BSON
import os
from tqdm import tqdm_notebook as tqdm
from subprocess import check_output
print(check_output(["ls", ".."]).decode("utf8"))

In [ ]:
if os.path.isfile("train_small.bson"):
    os.remove("train_small.bson")

if os.path.isfile("test_small.bson"):
    os.remove("test_small.bson")
    
f = open('train_small.bson', 'ab+')

MAX_TRAIN_FILE_SIZE = 40 * 1024 * 1024
MAX_TEST_FILE_SIZE = 5 * 1024 * 1024

data = bson.decode_file_iter(open('../input/train.bson', 'rb'))
cat_list = list()
pbar = tqdm(total=MAX_TRAIN_FILE_SIZE)
sz_last = 0
for d in data:
    category_id = d['category_id']
    if not category_id in cat_list:
        f.write(BSON.encode(d))
        cat_list.append(category_id)        
        fsz = f.tell()
        sz_diff = fsz - sz_last
        pbar.update(sz_diff)
        sz_last = fsz
        if fsz > MAX_TRAIN_FILE_SIZE:
            break
pbar.close()
f.close()
print("Saved {:d} categories in train.".format(len(cat_list)))

## Saving test sample as well.
f = open('test_small.bson', 'ab+')
data = bson.decode_file_iter(open('../input/test.bson', 'rb'))
pbar = tqdm(total=MAX_TEST_FILE_SIZE)
sz_last = 0
num_samples = 0
for d in data:
    f.write(BSON.encode(d))
    num_samples += 1
    fsz = f.tell()
    sz_diff = fsz - sz_last
    pbar.update(sz_diff)
    sz_last = fsz
    if fsz > MAX_TEST_FILE_SIZE:
        break
pbar.close()
f.close()
print("Saved {:d} samples in test.".format(num_samples))